# File Name: advanced_image_patterns_part3.ipynb
### Location: Chapter 18
### Purpose: 
#####       1. Image model customization

##### Dependency: simple-sageMaker-bedrock.ipynb at Chapter 3 should work properly.
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%%time 

%pip install --no-build-isolation --force-reinstall -q \
    "boto3" \
    "awscli" \
    "botocore" \
    "utils" \
    "matplotlib" \
    "sagemaker" \
    "numpy<2"

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import os
import sys
import json
import io
import base64
import random
import warnings
import boto3
import botocore
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from PIL import Image, ImageOps
import sagemaker

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

## Define important environment variable

In [ ]:
# Try-except block to handle potential errors during execution
try:
    # Create a new Boto3 session to interact with AWS services
    # This session manages credentials and region configuration for AWS interactions
    boto3_session = boto3.session.Session()

    # Retrieve the current AWS region from the session (e.g., 'us-east-1', 'us-west-2')
    aws_region_name = boto3_session.region_name

    # Initialize Bedrock and Bedrock Runtime clients using Boto3
    # These clients enable interactions with AWS Bedrock-related services
    boto3_bedrock_client = boto3.client('bedrock', region_name=aws_region_name)
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=aws_region_name)

    # Create a SageMaker session and retrieve the execution role ARN
    # The role ARN authorizes SageMaker to perform tasks on behalf of the user
    sagemaker_role_arn = sagemaker.get_execution_role()

    # Specify the Amazon Titan image generator model ID for multimodal processing
    amazon_titan_image_model_id = "amazon.titan-image-generator-v2:0"

    # Specify the Amazon Titan embedding model ID for multimodal indexing
    multimodal_embed_model_id = "amazon.titan-embed-image-v1"

    # Store all relevant variables in a dictionary for easier access and management
    variables_store = {
        "aws_region_name": aws_region_name,                           # AWS region name
        "boto3_bedrock_client": boto3_bedrock_client,                 # Bedrock client instance
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client, # Bedrock Runtime client instance
        "boto3_session": boto3_session,                               # Current Boto3 session object
        "sagemaker_role_arn": sagemaker_role_arn,                     # SageMaker execution role ARN
        "multimodal_embed_model_id": multimodal_embed_model_id,       # Titan embedding model ID
        "amazon_titan_image_model_id": amazon_titan_image_model_id    # Titan image generator model ID
    }

    # Print all stored variables for debugging and verification
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

# Handle any exceptions that occur during the execution
except Exception as e:
    # Print an error message if an unexpected error occurs
    print(f"An unexpected error occurred: {e}")


# Common code 

### The provided Python code allows you to plot multiple images side by side with corresponding headings.

In [ ]:
%%time
def plot_multiple_images(images, headings, cols=3):
    """
    Plots multiple images side by side with headings.

    Parameters:
    images (list): A list of image paths.
    headings (list): A list of headings for each image.
    cols (int): The number of columns to display the images in (default is 3).
    """
    # Calculate number of rows needed
    rows = (len(images) + cols - 1) // cols

    # Create a figure and axes
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))

    # Flatten axes array if there are multiple rows
    axes = axes.flatten()

    # Loop through images and plot them
    for i in range(len(images)):
        # Read the image from file path
        img = mpimg.imread(images[i])
        
        # Plot the image
        axes[i].imshow(img)
        axes[i].axis('off')  # Hide axis
        
        # Add the heading
        axes[i].set_title(headings[i], fontsize=14, weight='bold')

    # Hide unused axes if the number of images is less than the number of axes
    for i in range(len(images), len(axes)):
        axes[i].axis('off')

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Example usage
# plot_multiple_images(images, headings, cols=2)

### base64 encoded string of the image

In [ ]:
%%time
def encode_image(image_path):
    # Open the image file in binary mode
    with open(image_path, "rb") as image_file:
        # Encode the image to base64
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_image

# Example usage
## encoded_image = encode_image(image_path)

### Decode base64 to image and save

In [ ]:
%%time

def decode_image(encoded_image_b64, save_path):
    """
    Decodes a base64-encoded image and saves it to a specified file location.

    Args:
        encoded_image_b64 (str): The base64-encoded string of the image.
        save_path (str): The path where the decoded image will be saved.

    Returns:
        None
    """
    try:
        # Decode the base64 image and open it as a PIL Image
        decoded_image = Image.open(
            io.BytesIO(
                base64.decodebytes(bytes(encoded_image_b64, "utf-8"))
            )
        )
        
        # Save the decoded image to the specified path
        decoded_image.save(save_path)
        print(f"Image successfully saved to {save_path}")
    except (base64.binascii.Error, IOError) as e:
        print(f"Error decoding or saving the image: {e}")
        raise  # Optionally re-raise the exception for external handling

# Example usage
# decode_image(generated_image_b64, "output_image.png")

### The function invoke_bedrock_model is designed to call an Amazon Bedrock model for image generation, with robust error handling and clear feedback mechanisms. It accepts a Boto3 runtime client, a JSON payload (body), and an optional model ID

In [ ]:
%%time
def invoke_bedrock_model(boto3_bedrock_runtime_client, body, model_id):
    """
    Invokes the Amazon Bedrock model for image generation with error handling.
    
    Args:
        boto3_bedrock_runtime_client: The Boto3 Bedrock runtime client.
        body (str): The request payload as a JSON string.
        model_id (str): The model identifier to invoke (default: "amazon.titan-image-generator-v1").
    
    Returns:
        str: The base64-encoded image data if successful.
    """
    try:
        # Model invocation
        response = boto3_bedrock_runtime_client.invoke_model(
            body=body,
            modelId=model_id,
            accept="application/json", 
            contentType="application/json"
        )
        
        # Output processing
        response_body = json.loads(response.get("body").read())  # Decode response body
        img_b64 = response_body["images"][0]  # Extract the first image (base64-encoded)
        print(f"Output (truncated): {img_b64[:80]}...")  # Print truncated base64 string for debugging
        
        return img_b64  # Return the full base64-encoded image
    
    except KeyError as e:
        print(f"KeyError: Missing expected key in response - {e}")
        return None
    
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: Failed to parse response body - {e}")
        return None
    
    except boto3.exceptions.Boto3Error as e:
        print(f"Boto3Error: AWS SDK error - {e}")
        return None
    
    except Exception as e:
        print(f"Unexpected Error: {e}")
        return None

# Section 10: Image model customization

##### You will explore 2 use cases below. 

##### Use Case 1: Generating Image Variations Using a Prompt and Reference Image(s)
##### Use Case 2: Instant Customization Through Style Transfer


##### similarityStrength (Optional) – Specifies how similar the generated image should be to the input images(s). Range in 0.2 to 1.0 with lower values used to introduce more randomness. Refer: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html

# Use Case 1: Generating Image Variations Using a Prompt and Reference Image(s)


### You previously generated an image of a river at dusk using a prompt. Now, you want to transfer the same referance with majestic Royal Bengal tiger.
### The original image is located at data/generated_image/generated_image_15.png.
### To achieve this, you will use the generating image variations using a prompt and reference image(s) technique.

# Prompt for generating image variations using a prompt and reference image(s)

In [ ]:
prompt = "Realistic portrait of a majestic Royal Bengal tiger in its natural habitat, with vibrant orange fur, black stripes, and piercing amber eyes."

#### The code demonstrates two use cases for generating images using a prompt and reference images. It defines paths for the source and target images, with and without references. The source image is read and encoded in base64 for reference-based generation. For the text-to-image generation without reference, a JSON body specifies parameters such as the prompt, number of images, quality, and seed. For the image variation with reference, another JSON body includes the encoded reference image, a similarity strength parameter (set to 0.70), and the same configuration for image quality and seed. These configurations are prepared to support generating a premium-quality image either entirely from text or by using an existing image for variation.

In [ ]:
%%time

use_case1_reference_source_image_path = "data/generated_image/generated_image_15.png"
use_case1_target_image_path_wo_ref = "data/generated_image/generated_image_tiger_wo_ref.png"
use_case1_target_image_path_w_ref = "data/generated_image/generated_image_tiger_w_ref.png"

# Read the image file and encode it in base64
source_encoded_image = encode_image(use_case1_reference_source_image_path)

## body for without referance
bodywo_ref = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams":{
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "premium",
            "seed": 5000
        }
    })

## body for with referance
bodyw_ref = json.dumps({
        "taskType": "IMAGE_VARIATION",
        "imageVariationParams": {
            "text": prompt,
            "images": [source_encoded_image], # can provide up to 5 reference images here
            "similarityStrength": 0.70,  # between 0.2 to 1
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "premium",
            "seed": 5000
        }
    })

In [ ]:
%%time
generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, bodywo_ref, amazon_titan_image_model_id )
decode_image(generated_image_b64, use_case1_target_image_path_wo_ref)

generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, bodyw_ref, amazon_titan_image_model_id )
decode_image(generated_image_b64, use_case1_target_image_path_w_ref)

print("\n\n")
images = [ use_case1_reference_source_image_path , use_case1_target_image_path_wo_ref , use_case1_target_image_path_w_ref ]
headings =[ "Source referance Image" , "Generated image with out referance" , "Generated image with referance" ]
plot_multiple_images(images, headings, cols=3)
print("\n\n")

# Use Case 2: Instant Customization Through Style Transfer


### You previously generated an image of an elephant in a beautifull landscape using a prompt. Now, you want to transfer the same style while generating image of majestic Royal Bengal tiger.
### The original image is located at data/generated_image/generated_image_5.png.
### To achieve this, you will use the generating image with style trasnfer technique.

# Prompt for style transfer

In [ ]:
prompt = "Realistic portrait of a majestic Royal Bengal tiger in its natural habitat, with vibrant orange fur, black stripes, and piercing amber eyes."

In [ ]:
%%time

use_case2_reference_source_image_path = "data/generated_image/generated_image_5.png"
use_case2_target_image_path_wo_ref = "data/generated_image/generated_image_elephant_wo_ref.png"
use_case2_target_image_path_w_ref = "data/generated_image/generated_image_elephant_w_ref.png"

# Read the image file and encode it in base64
source_encoded_image = encode_image(use_case2_reference_source_image_path)

## body for without referance
bodywo_ref = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams":{
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "premium",
            "seed": 500
        }
    })

## body for with referance
bodyw_ref = json.dumps({
        "taskType": "IMAGE_VARIATION",
        "imageVariationParams": {
            "text": prompt,
            "images": [source_encoded_image], # can provide up to 5 reference images here
            "similarityStrength": 0.75,  # between 0.2 to 1
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "premium",
            "seed": 500
        }
    })

In [ ]:
%%time
generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, bodywo_ref, amazon_titan_image_model_id )
decode_image(generated_image_b64, use_case2_target_image_path_wo_ref)

generated_image_b64 = invoke_bedrock_model(boto3_bedrock_runtime_client, bodyw_ref, amazon_titan_image_model_id )
decode_image(generated_image_b64, use_case2_target_image_path_w_ref)

print("\n\n")
images = [ use_case2_reference_source_image_path , use_case2_target_image_path_wo_ref , use_case2_target_image_path_w_ref ]
headings =[ "Source referance Image" , "Generated image with out referance" , "Generated image with referance" ]
plot_multiple_images(images, headings, cols=3)
print("\n\n")

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html